In [1]:
import numpy as np
import time
from lidsor_filter import filtering_lidsor_cpp
from glob import glob
import open3d as o3d
import pypatchworkpp

In [6]:
def segment_and_filter_points(points, scaling_factor=110.0):
    """
    Segment ground points and filter non-ground points using LIDSOR.

    Args:
        points (np.ndarray): Input point cloud data of shape (N, 4) containing x, y, z, intensity
        scaling_factor (float): Scaling factor for intensity values

    Returns:
        np.ndarray: Merged point cloud containing filtered ground and non-ground points
    """
    # Ground point estimation
    params = pypatchworkpp.Parameters()
    patchwork = pypatchworkpp.patchworkpp(params)
    patchwork.estimateGround(points)

    # Get ground and non-ground indices
    ground_idx = patchwork.getGroundIndices()
    nonground_idx = patchwork.getNongroundIndices()

    # Split points
    ground_points = points[ground_idx]
    nonground_points = points[nonground_idx]

    # Filter non-ground points
    filtered_points, kept_indices, removed_indices = filtering_lidsor_cpp(
        nonground_points, scaling_factor=scaling_factor
    )
    filtered_points = np.array(filtered_points, dtype=np.float64)

    # Merge ground and filtered non-ground points
    merged_points = np.concatenate([ground_points, filtered_points], axis=0)

    return merged_points, kept_indices, removed_indices

In [3]:
bin_path_list = glob(f"/Users/yukiya/ws/mini/*.bin")
bin_path = bin_path_list[0]


points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
print(points[:, 3])

[0.         0.         0.         ... 0.13636364 0.2090909  0.27272728]


In [34]:
# 反射率によるフィルタリング，lidsor，lidsor+patchworkによる比較
intensity_threshold = 0.0

def intensity_filter(points, intensity_threshold):
    kept_indices = np.where(points[:, 3] <= intensity_threshold)[0]
    removed_indices = np.where(points[:, 3] > intensity_threshold)[0]
    return points[kept_indices], kept_indices, removed_indices

org_points = []
intensity_filtered_points = []
lidsor_points = []
lidsor_patchwork_points = []

for n in range(len(bin_path_list)):
    points = np.fromfile(bin_path_list[n], dtype=np.float32).reshape(-1, 4)
    org_points.append(points)
    intensity_filtered_points.append(intensity_filter(points, intensity_threshold))
    lidsor_points.append(filtering_lidsor_cpp(points, scaling_factor=110.0))

    lidsor_patchwork_points.append(segment_and_filter_points(points, scaling_factor=110.0))


PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
PatchWorkpp::PatchWorkpp() - INITIALIZATION CO

In [37]:
visualize_point_cloud(intensity_filtered_points[0][0])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [43]:
# それぞれの可視化
def visualize_point_cloud(points, removed_indices=None, window_name="Point Cloud"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    
    # 高さ（z座標）に基づいて色を計算
    z_values = points[:, 2]
    z_min, z_max = np.min(z_values), np.max(z_values)
    normalized_z = (z_values - z_min) / (z_max - z_min)
    
    # カラーマップの作成（例：青から赤へのグラデーション）
    colors = np.zeros((points.shape[0], 3))
    colors[:, 2] = 1 - normalized_z  # 青成分
    colors[:, 0] = normalized_z      # 赤成分
    
    # removed_indicesの点を黒色に設定
    if removed_indices is not None:
        colors[removed_indices] = [0, 0, 0]
    
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Create a separate window and visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name)
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

# Visualize intensity filtered points
n = 0  # Index of the frame to visualize
visualize_point_cloud(
    org_points[n], 
    intensity_filtered_points[n][2],
    window_name="Intensity Filtered"
)

visualize_point_cloud(
    org_points[n], 
    lidsor_points[n][2],
    window_name="LIDSOR"
)

# # Visualize Patchwork+LIDSOR results
# points = np.fromfile(bin_path_list[n], dtype=np.float32).reshape(-1, 4)
# merged_points, kept_indices, removed_indices = lidsor_patchwork_points[n]
# visualize_point_cloud(
#     merged_points,
#     removed_indices,
#     window_name="Patchwork + LIDSOR"
# )

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [8]:

ground_idx = PatchworkPLUSPLUS.getGroundIndices()
ground_pcd = o3d.geometry.PointCloud()

ground_intensity = points[ground_idx][:, 3]

# Create colors array, default to white
colors = np.ones((ground_intensity.shape[0], 3))

# Set points with intensity below threshold to black
threshold = 0.01 / 110.0  # Set your desired threshold here
colors[ground_intensity <= threshold] = [0, 0, 0]  # Black color for low intensity

ground_pcd.points = o3d.utility.Vector3dVector(points[ground_idx][:, :3])
ground_pcd.colors = o3d.utility.Vector3dVector(colors)  # Assign colors to the point cloud
# o3d.visualization.draw_geometries([ground_pcd])


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
